# Stochastic Oscillator - Complete Trading Tutorial

## Overview
This notebook provides a comprehensive tutorial on the Stochastic Oscillator indicator, including:
- Concept and theory
- Mathematical formula and calculation
- Manual implementation in Python
- TradingView integration
- Binance platform usage
- Practical trading strategies
- Backtesting examples

## 1. Introduction to Stochastic Oscillator

### What is Stochastic Oscillator?
The Stochastic Oscillator is a momentum indicator that compares a particular closing price of a security to a range of its prices over a certain period of time. Developed by George Lane in the late 1950s, it oscillates between 0 and 100 and is used to identify overbought and oversold conditions.

### Why Use Stochastic Oscillator?
- **Momentum Measurement**: Measures the speed and direction of price movements
- **Overbought/Oversold**: Identifies extreme price levels
- **Divergence**: Detects potential trend reversals
- **Trend Confirmation**: Confirms the strength of existing trends

### Best Timeframes and Markets
- **Short-term**: Stochastic(5, 3, 3) for day trading
- **Medium-term**: Stochastic(14, 3, 3) for swing trading
- **Long-term**: Stochastic(21, 5, 5) for position trading
- **Markets**: Works well in both trending and ranging markets

## 2. Mathematical Formula

### Stochastic Oscillator Formula
```
%K = 100 * (Current Close - Lowest Low) / (Highest High - Lowest Low)

%D = 3-day SMA of %K

Where:
- %K = Fast stochastic line
- %D = Slow stochastic line (signal line)
- Highest High = Highest high over the lookback period
- Lowest Low = Lowest low over the lookback period
```

### Parameters:
- **%K Period**: Number of periods for %K calculation (typically 14)
- **%D Period**: Number of periods for %D smoothing (typically 3)
- **Slowing**: Additional smoothing factor (typically 3)
- **Overbought**: Typically 80
- **Oversold**: Typically 20

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta
import sys
sys.path.append('../utils')

# Import utilities
from data_downloader import DataDownloader
from indicators import IndicatorCalculator
from backtest_engine import BacktestEngine, StrategyGenerator

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

## 3. Data Download

Let's download historical crypto data for our analysis.

In [ ]:
# Initialize data downloader
downloader = DataDownloader()

# Download BTC/USDT data
pair = "BTC/USDT"
timeframe = "1d"
days = 365

print(f"Downloading {pair} data for {days} days...")
df = downloader.get_market_data(pair, timeframe, days, source='ccxt')

if df is not None:
    print(f"\nDownloaded {len(df)} candles of {pair} data")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")
    print(f"\nFirst few rows:")
    display(df.head())
else:
    print("Failed to download data")

## 4. Manual Calculation of Stochastic Oscillator

Let's implement Stochastic Oscillator from scratch to understand how it works.

In [ ]:
def calculate_stochastic_manual(data, k_period=14, d_period=3, slowing=3):
    """
    Calculate Stochastic Oscillator manually
    """
    df = data.copy()
    
    # Calculate highest high and lowest low over K period
    df['highest_high'] = df['high'].rolling(window=k_period).max()
    df['lowest_low'] = df['low'].rolling(window=k_period).min()
    
    # Calculate %K
    df['k_raw'] = 100 * (df['close'] - df['lowest_low']) / (df['highest_high'] - df['lowest_low'])
    
    # Apply slowing to %K
    df['k'] = df['k_raw'].rolling(window=slowing).mean()
    
    # Calculate %D (SMA of %K)
    df['d'] = df['k'].rolling(window=d_period).mean()
    
    return df[['k', 'd']]

# Calculate Stochastic Oscillator manually
df_manual = df.copy()
stoch_manual = calculate_stochastic_manual(df_manual, k_period=14, d_period=3, slowing=3)
df_manual['k_14'] = stoch_manual['k']
df_manual['d_14'] = stoch_manual['d']

print("Stochastic Oscillator calculated manually:")
display(df_manual[['close', 'k_14', 'd_14']].tail(10))

# Let's verify the calculation manually
print("\nManual verification of Stochastic(14, 3, 3) for the last row:")
recent_data = df_manual.tail(15)  # 14 periods + 1 for calculation
highest_high = recent_data['high'].max()
lowest_low = recent_data['low'].min()
current_close = recent_data['close'].iloc[-1]

k_raw = 100 * (current_close - lowest_low) / (highest_high - lowest_low)
manual_k = k_raw  # No slowing for single calculation
calculated_k = df_manual['k_14'].iloc[-1]
calculated_d = df_manual['d_14'].iloc[-1]

print(f"Highest High: {highest_high:.2f}")
print(f"Lowest Low: {lowest_low:.2f}")
print(f"Current Close: {current_close:.2f}")
print(f"Manual %K: {manual_k:.2f}")
print(f"Calculated %K: {calculated_k:.2f}")
print(f"Calculated %D: {calculated_d:.2f}")
print(f"Difference: {abs(manual_k - calculated_k):.6f}")

## 5. Library Implementation

Now let's use established libraries to calculate Stochastic Oscillator.

In [ ]:
# Using our utilities
df_utils = df.copy()
df_utils['k_14_utils'], df_utils['d_14_utils'] = IndicatorCalculator.stochastic(
    df_utils['high'], df_utils['low'], df_utils['close'], k_period=14, d_period=3
)

# Using pandas-ta
import pandas_ta as ta
df_ta = df.copy()
df_ta.ta.stoch(high='high', low='low', close='close', k=14, d=3, append=True)

# Using TA-Lib
try:
    import talib
    df_talib = df.copy()
    df_talib['k_14_talib'], df_talib['d_14_talib'] = talib.STOCH(
        df_talib['high'].values, df_talib['low'].values, df_talib['close'].values,
        fastk_period=14, slowk_period=3, slowd_period=3
    )
    print("TA-Lib Stochastic calculated successfully")
except ImportError:
    print("TA-Lib not installed, skipping TA-Lib example")
    df_talib = df.copy()
    df_talib['k_14_talib'] = np.nan
    df_talib['d_14_talib'] = np.nan

# Compare results
print("Comparison of different Stochastic implementations:")
comparison = pd.DataFrame({
    'Manual_K': df_manual['k_14'],
    'Manual_D': df_manual['d_14'],
    'Utils_K': df_utils['k_14_utils'],
    'Utils_D': df_utils['d_14_utils'],
    'Pandas-TA_K': df_ta['STOCHk_14_3_3'],
    'Pandas-TA_D': df_ta['STOCHd_14_3_3'],
    'TA-Lib_K': df_talib['k_14_talib'],
    'TA-Lib_D': df_talib['d_14_talib']
})

display(comparison.dropna().tail(10))

# Check if all methods produce similar results
print("\nDifferences between methods:")
print(f"Manual vs Utils K max difference: {abs(comparison['Manual_K'] - comparison['Utils_K']).max():.8f}")
print(f"Manual vs Utils D max difference: {abs(comparison['Manual_D'] - comparison['Utils_D']).max():.8f}")
print(f"Manual vs Pandas-TA K max difference: {abs(comparison['Manual_K'] - comparison['Pandas-TA_K']).max():.8f}")

## 6. Visualization

Let's visualize Stochastic Oscillator with price action.

In [ ]:
# Create interactive plot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(f'{pair} - Price Action', f'{pair} - Stochastic Oscillator'),
    vertical_spacing=0.1,
    row_heights=[0.6, 0.4]
)

# Price chart
fig.add_trace(
    go.Candlestick(
        x=df_manual.index,
        open=df_manual['open'],
        high=df_manual['high'],
        low=df_manual['low'],
        close=df_manual['close'],
        name='Price'
    ),
    row=1, col=1
)

# %K line
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['k_14'],
        mode='lines',
        name='%K (14)',
        line=dict(color='blue', width=2)
    ),
    row=2, col=1
)

# %D line
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['d_14'],
        mode='lines',
        name='%D (3)',
        line=dict(color='red', width=2)
    ),
    row=2, col=1
)

# Overbought/Oversold levels
fig.add_hline(y=80, line=dict(color='red', dash='dash'), annotation_text='Overbought (80)', row=2, col=1)
fig.add_hline(y=20, line=dict(color='green', dash='dash'), annotation_text='Oversold (20)', row=2, col=1)
fig.add_hline(y=50, line=dict(color='gray', dash='dot'), annotation_text='Midline (50)', row=2, col=1)

# Highlight overbought/oversold areas
fig.add_trace(
    go.Scatter(
        x=df_manual.index,
        y=df_manual['k_14'],
        mode='markers',
        marker=dict(
            size=4,
            color=np.where(df_manual['k_14'] > 80, 'red', 
                         np.where(df_manual['k_14'] < 20, 'green', 'blue'))
        ),
        name='%K Levels',
        showlegend=False
    ),
    row=2, col=1
)

fig.update_layout(
    title=f'{pair} - Stochastic Oscillator',
    template='plotly_dark',
    height=800,
    xaxis_rangeslider_visible=False
)

fig.show()

# Create static plot showing Stochastic characteristics
plt.figure(figsize=(15, 10))

# Price and Stochastic
plt.subplot(2, 1, 1)
plt.plot(df_manual.index, df_manual['close'], label='Price', alpha=0.7, color='black')
plt.title(f'{pair} - Price Action')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_manual.index, df_manual['k_14'], label='%K (14)', alpha=0.8, color='blue')
plt.plot(df_manual.index, df_manual['d_14'], label='%D (3)', alpha=0.8, color='red')
plt.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='Overbought (80)')
plt.axhline(y=20, color='green', linestyle='--', alpha=0.5, label='Oversold (20)')
plt.axhline(y=50, color='gray', linestyle=':', alpha=0.5, label='Midline (50)')

# Color background based on Stochastic levels
plt.fill_between(df_manual.index, 20, 80, alpha=0.1, color='gray', label='Neutral Zone')
plt.fill_between(df_manual.index, 80, 100, alpha=0.1, color='red', label='Overbought Zone')
plt.fill_between(df_manual.index, 0, 20, alpha=0.1, color='green', label='Oversold Zone')

plt.title('Stochastic Oscillator')
plt.xlabel('Date')
plt.ylabel('Stochastic')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

## 7. TradingView Integration

### Pine Script for Stochastic Oscillator

```pine
// Stochastic Oscillator Pine Script
//@version=5
indicator("Stochastic Oscillator", shorttitle="Stoch", format=format.price, precision=2)

// Input parameters
kPeriod = input.int(14, title="%K Period", minval=1)
dPeriod = input.int(3, title="%D Period", minval=1)
slowing = input.int(3, title="Slowing", minval=1)
overbought = input.int(80, "Overbought", minval=50, maxval=100)
oversold = input.int(20, "Oversold", minval=0, maxval=50)

// Calculate Stochastic
highestHigh = ta.highest(high, kPeriod)
lowestLow = ta.lowest(low, kPeriod)
kRaw = 100 * (close - lowestLow) / (highestHigh - lowestLow)
k = ta.sma(kRaw, slowing)
d = ta.sma(k, dPeriod)

// Plot Stochastic
plot(k, "%K", color=color.new(color.blue, 0))
plot(d, "%D", color=color.new(color.red, 0))
band1 = hline(overbought, "Overbought", color=color.new(color.red, 50))
band0 = hline(oversold, "Oversold", color=color.new(color.green, 50))
fill(band1, band0, color=color.new(color.gray, 90), title="Middle Band")

// Add signals
buySignal = ta.crossover(k, oversold)
sellSignal = ta.crossunder(k, overbought)

plotshape(buySignal, title="Buy Signal", location=location.belowbar, color=color.green, style=shape.triangleup, size=size.small)
plotshape(sellSignal, title="Sell Signal", location=location.abovebar, color=color.red, style=shape.triangledown, size=size.small)

// Crossover signals
crossUp = ta.crossover(k, d)
crossDown = ta.crossunder(k, d)

plotshape(crossUp, title="Bullish Cross", location=location.belowbar, color=color.lime, style=shape.circle, size=size.tiny)
plotshape(crossDown, title="Bearish Cross", location=location.abovebar, color=color.red, style=shape.circle, size=size.tiny)
```

### How to Add to TradingView:
1. Open TradingView chart
2. Click on "Pine Editor" tab at the bottom
3. Copy and paste the script above
4. Click "Add to Chart"
5. Adjust parameters in the settings

### TradingView Built-in Stochastic:
1. Click on "Indicators" at the top
2. Search for "Stochastic Oscillator"
3. Select "Stochastic" from the list
4. Adjust parameters and overbought/oversold levels

## 8. Binance Integration

### How to Add Stochastic Oscillator on Binance:
1. Open Binance trading interface
2. Select your trading pair (e.g., BTC/USDT)
3. Click on "Indicators" at the top of the chart
4. Search for "Stochastic Oscillator"
5. Select "Stochastic" from the list
6. In the settings:
   - Set "%K Period" (default: 14)
   - Set "%D Period" (default: 3)
   - Adjust overbought/oversold levels
   - Choose colors and line styles
7. Click "Apply"

### Binance Stochastic Parameters:
- **%K Period**: 1-500 (common values: 5, 14, 21)
- **%D Period**: 1-500 (common values: 3, 5)
- **Slowing**: 1-500 (common values: 3, 5)
- **Source**: Close, Open, High, Low, HL/2, HLC/3, OHLC/4
- **Overbought**: Typically 80 or 90
- **Oversold**: Typically 20 or 10
- **Colors**: Custom color selection for %K and %D lines

## 9. Trading Strategies

### Strategy 1: Stochastic Overbought/Oversold
**Description**: Buy when Stochastic is oversold, sell when Stochastic is overbought

**Rules**:
- **Entry**: %K crosses below oversold level (e.g., 20)
- **Exit**: %K crosses above overbought level (e.g., 80)
- **Stop Loss**: 2-3% below entry price
- **Take Profit**: 5-10% above entry price or when opposite signal occurs

In [ ]:
def stochastic_overbought_oversold_strategy(data, k_period=14, d_period=3, slowing=3, oversold=20, overbought=80):
    """
    Stochastic Overbought/Oversold Strategy
    """
    df = data.copy()
    
    # Calculate Stochastic
    df['k'], df['d'] = IndicatorCalculator.stochastic(
        df['high'], df['low'], df['close'], k_period, d_period
    )
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['k'] <= oversold, 'signal'] = 1  # Buy when oversold
    df.loc[df['k'] >= overbought, 'signal'] = -1  # Sell when overbought
    
    # Identify crossovers
    df['k_oversold_cross'] = ((df['k'] <= oversold) & 
                               (df['k'].shift(1) > oversold))
    df['k_overbought_cross'] = ((df['k'] >= overbought) & 
                                (df['k'].shift(1) < overbought))
    
    return df

# Apply strategy
df_stoch_oo = stochastic_overbought_oversold_strategy(df_manual)

# Plot signals
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(f'{pair} - Stochastic Overbought/Oversold Strategy', f'{pair} - Stochastic Oscillator'),
    vertical_spacing=0.1,
    row_heights=[0.6, 0.4]
)

# Price chart
fig.add_trace(
    go.Candlestick(
        x=df_stoch_oo.index,
        open=df_stoch_oo['open'],
        high=df_stoch_oo['high'],
        low=df_stoch_oo['low'],
        close=df_stoch_oo['close'],
        name='Price'
    ),
    row=1, col=1
)

# Buy signals
buy_signals = df_stoch_oo[df_stoch_oo['k_oversold_cross']]
fig.add_trace(
    go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        marker=dict(symbol='triangle-up', size=12, color='green'),
        name='Buy Signal'
    ),
    row=1, col=1
)

# Sell signals
sell_signals = df_stoch_oo[df_stoch_oo['k_overbought_cross']]
fig.add_trace(
    go.Scatter(
        x=sell_signals.index,
        y=sell_signals['close'],
        mode='markers',
        marker=dict(symbol='triangle-down', size=12, color='red'),
        name='Sell Signal'
    ),
    row=1, col=1
)

# Stochastic chart
fig.add_trace(
    go.Scatter(
        x=df_stoch_oo.index,
        y=df_stoch_oo['k'],
        mode='lines',
        name='%K (14)',
        line=dict(color='blue', width=2)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_stoch_oo.index,
        y=df_stoch_oo['d'],
        mode='lines',
        name='%D (3)',
        line=dict(color='red', width=2)
    ),
    row=2, col=1
)

# Overbought/Oversold levels
fig.add_hline(y=80, line=dict(color='red', dash='dash'), row=2, col=1)
fig.add_hline(y=20, line=dict(color='green', dash='dash'), row=2, col=1)

fig.update_layout(
    title=f'{pair} - Stochastic Overbought/Oversold Strategy',
    template='plotly_dark',
    height=800
)

fig.show()

print(f"Total buy signals: {len(buy_signals)}")
print(f"Total sell signals: {len(sell_signals)}")
print(f"\nRecent signals:")
recent_signals = df_stoch_oo[df_stoch_oo['k_oversold_cross'] | df_stoch_oo['k_overbought_cross']].tail(10)
display(recent_signals[['close', 'k', 'd', 'k_oversold_cross', 'k_overbought_cross']])

### Strategy 2: Stochastic Crossover
**Description**: Trade based on crossovers between %K and %D lines

**Rules**:
- **Bullish Signal**: %K crosses above %D
- **Bearish Signal**: %K crosses below %D
- **Entry**: When crossover occurs
- **Exit**: When opposite crossover occurs or at extreme levels

In [ ]:
def stochastic_crossover_strategy(data, k_period=14, d_period=3, slowing=3):
    """
    Stochastic Crossover Strategy
    """
    df = data.copy()
    
    # Calculate Stochastic
    df['k'], df['d'] = IndicatorCalculator.stochastic(
        df['high'], df['low'], df['close'], k_period, d_period
    )
    
    # Generate signals
    df['signal'] = 0
    
    # Bullish crossover: %K crosses above %D
    df['bullish_cross'] = (df['k'] > df['d']) & (df['k'].shift(1) <= df['d'].shift(1))
    
    # Bearish crossover: %K crosses below %D
    df['bearish_cross'] = (df['k'] < df['d']) & (df['k'].shift(1) >= df['d'].shift(1))
    
    df.loc[df['bullish_cross'], 'signal'] = 1
    df.loc[df['bearish_cross'], 'signal'] = -1
    
    return df

# Apply strategy
df_stoch_cross = stochastic_crossover_strategy(df_manual)

# Create signals DataFrame for backtesting
stoch_cross_signals = pd.DataFrame(index=df_stoch_cross.index)
stoch_cross_signals['signal'] = df_stoch_cross['signal']

print(f"Bullish crossovers detected: {df_stoch_cross['bullish_cross'].sum()}")
print(f"Bearish crossovers detected: {df_stoch_cross['bearish_cross'].sum()}")

# Plot crossovers
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(df_stoch_cross.index, df_stoch_cross['close'], label='Price', alpha=0.7, color='black')
plt.scatter(df_stoch_cross[df_stoch_cross['bullish_cross']].index, 
            df_stoch_cross[df_stoch_cross['bullish_cross']]['close'], 
            color='green', s=100, marker='^', label='Bullish Cross', zorder=5)
plt.scatter(df_stoch_cross[df_stoch_cross['bearish_cross']].index, 
            df_stoch_cross[df_stoch_cross['bearish_cross']]['close'], 
            color='red', s=100, marker='v', label='Bearish Cross', zorder=5)
plt.title(f'{pair} - Stochastic Crossover Strategy')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_stoch_cross.index, df_stoch_cross['k'], label='%K (14)', alpha=0.8, color='blue')
plt.plot(df_stoch_cross.index, df_stoch_cross['d'], label='%D (3)', alpha=0.8, color='red')
plt.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='Overbought (80)')
plt.axhline(y=20, color='green', linestyle='--', alpha=0.5, label='Oversold (20)')
plt.title('Stochastic Oscillator')
plt.xlabel('Date')
plt.ylabel('Stochastic')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

### Strategy 3: Stochastic with Divergence
**Description**: Trade based on divergence between price and Stochastic Oscillator

**Rules**:
- **Bullish Divergence**: Price makes lower lows while Stochastic makes higher lows
- **Bearish Divergence**: Price makes higher highs while Stochastic makes lower highs
- **Entry**: When divergence is confirmed
- **Exit**: When Stochastic reaches opposite extreme or divergence pattern completes

In [ ]:
def detect_stochastic_divergence(data, k_period=14, d_period=3, slowing=3, lookback=5):
    """
    Detect Stochastic divergence patterns
    """
    df = data.copy()
    
    # Calculate Stochastic
    df['k'], df['d'] = IndicatorCalculator.stochastic(
        df['high'], df['low'], df['close'], k_period, d_period
    )
    
    # Find local minima and maxima
    df['price_low'] = df['close'].rolling(window=lookback, center=True).min()
    df['price_high'] = df['close'].rolling(window=lookback, center=True).max()
    df['stoch_low'] = df['k'].rolling(window=lookback, center=True).min()
    df['stoch_high'] = df['k'].rolling(window=lookback, center=True).max()
    
    # Detect divergences
    df['bullish_divergence'] = False
    df['bearish_divergence'] = False
    
    for i in range(lookback, len(df) - lookback):
        # Bullish divergence: lower price low, higher Stochastic low
        if (df['price_low'].iloc[i] < df['price_low'].iloc[i-lookback] and
            df['stoch_low'].iloc[i] > df['stoch_low'].iloc[i-lookback]):
            df.iloc[i, df.columns.get_loc('bullish_divergence')] = True
        
        # Bearish divergence: higher price high, lower Stochastic high
        if (df['price_high'].iloc[i] > df['price_high'].iloc[i-lookback] and
            df['stoch_high'].iloc[i] < df['stoch_high'].iloc[i-lookback]):
            df.iloc[i, df.columns.get_loc('bearish_divergence')] = True
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['bullish_divergence'], 'signal'] = 1
    df.loc[df['bearish_divergence'], 'signal'] = -1
    
    return df

# Apply strategy
df_stoch_div = detect_stochastic_divergence(df_manual)

# Create signals DataFrame for backtesting
stoch_div_signals = pd.DataFrame(index=df_stoch_div.index)
stoch_div_signals['signal'] = df_stoch_div['signal']

print(f"Bullish divergences detected: {df_stoch_div['bullish_divergence'].sum()}")
print(f"Bearish divergences detected: {df_stoch_div['bearish_divergence'].sum()}")

# Plot divergences
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(df_stoch_div.index, df_stoch_div['close'], label='Price', alpha=0.7, color='black')
plt.scatter(df_stoch_div[df_stoch_div['bullish_divergence']].index, 
            df_stoch_div[df_stoch_div['bullish_divergence']]['close'], 
            color='green', s=100, marker='^', label='Bullish Divergence', zorder=5)
plt.scatter(df_stoch_div[df_stoch_div['bearish_divergence']].index, 
            df_stoch_div[df_stoch_div['bearish_divergence']]['close'], 
            color='red', s=100, marker='v', label='Bearish Divergence', zorder=5)
plt.title(f'{pair} - Stochastic Divergence Strategy')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(df_stoch_div.index, df_stoch_div['k'], label='%K (14)', alpha=0.8, color='blue')
plt.plot(df_stoch_div.index, df_stoch_div['d'], label='%D (3)', alpha=0.8, color='red')
plt.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='Overbought (80)')
plt.axhline(y=20, color='green', linestyle='--', alpha=0.5, label='Oversold (20)')
plt.title('Stochastic Oscillator')
plt.xlabel('Date')
plt.ylabel('Stochastic')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

## 10. Backtesting Results

Let's backtest our strategies and analyze the results.

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(initial_capital=10000)

# Create signals for each strategy
strategies = {
    'Stochastic_Overbought_Oversold': df_stoch_oo[['signal']],
    'Stochastic_Crossover': stoch_cross_signals,
    'Stochastic_Divergence': stoch_div_signals
}

# Run backtests
for name, signals in strategies.items():
    print(f"Backtesting {name} strategy...")
    engine.run_backtest(df_manual, signals, name)

# Compare strategies
comparison = engine.compare_strategies(list(strategies.keys()))
print("\nStrategy Comparison:")
display(comparison)

# Plot equity curves
engine.plot_equity_curve()

# Plot drawdowns
engine.plot_drawdown()

In [ ]:
# Generate detailed report for the best strategy
best_strategy = comparison['total_return'].idxmax()
print(f"Best performing strategy: {best_strategy}")
print(engine.generate_report(best_strategy))

In [ ]:
# Analyze trade statistics for the best strategy
best_data = engine.results[best_strategy]['data']

# Analyze Stochastic levels at trade entries
trade_analysis = best_data[best_data['signal'] != 0].copy()
trade_analysis['k'] = df_manual['k_14']
trade_analysis['d'] = df_manual['d_14']

if len(trade_analysis) > 0:
    print(f"\nTrade Analysis for {best_strategy}:")
    print(f"Total trades: {len(trade_analysis)}")
    print(f"Buy trades: {len(trade_analysis[trade_analysis['signal'] == 1])}")
    print(f"Sell trades: {len(trade_analysis[trade_analysis['signal'] == -1])}")
    print(f"Average %K at entry: {trade_analysis['k'].mean():.1f}")
    print(f"Average %D at entry: {trade_analysis['d'].mean():.1f}")
    print(f"%K range at entry: {trade_analysis['k'].min():.1f} - {trade_analysis['k'].max():.1f}")
    
    # Stochastic distribution at trade entries
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(trade_analysis['k'], bins=20, alpha=0.7, edgecolor='black', label='%K')
    plt.hist(trade_analysis['d'], bins=20, alpha=0.7, edgecolor='black', label='%D')
    plt.axvline(x=20, color='green', linestyle='--', label='Oversold (20)')
    plt.axvline(x=80, color='red', linestyle='--', label='Overbought (80)')
    plt.axvline(x=50, color='gray', linestyle=':', label='Midline (50)')
    plt.title('Stochastic Distribution at Trade Entries')
    plt.xlabel('Stochastic')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Performance by Stochastic zone
    trade_analysis['stoch_zone'] = pd.cut(trade_analysis['k'], 
                                        bins=[0, 20, 50, 80, 100], 
                                        labels=['Oversold', 'Low', 'High', 'Overbought'])
    
    plt.subplot(1, 2, 2)
    zone_performance = trade_analysis.groupby('stoch_zone')['strategy_returns'].mean() * 100
    zone_performance.plot(kind='bar', color=['green', 'blue', 'orange', 'red'])
    plt.title('Average Returns by Stochastic Zone')
    plt.xlabel('Stochastic Zone')
    plt.ylabel('Average Return (%)')
    plt.axhline(y=0, color='black', linestyle='--')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades completed in the backtest period")

## 11. Pros and Cons

### Advantages:
- ✅ **Leading Indicator**: Can signal trend changes before they occur
- ✅ **Overbought/Oversold**: Clear levels for extreme conditions
- ✅ **Divergence**: Early warning of potential reversals
- ✅ **Crossover Signals**: Clear buy/sell signals from %K/%D crossovers
- ✅ **Versatile**: Works in various market conditions

### Disadvantages:
- ❌ **False Signals**: Can remain overbought/oversold for extended periods
- ❌ **Whipsaw**: Can generate false signals in choppy markets
- ❌ **Lagging**: Based on historical price data
- ❌ **Parameter Sensitivity**: Results can vary significantly with different parameters

### When to Use:
- **Ranging Markets**: Excellent for sideways price action
- **Reversal Points**: Good for identifying potential turning points
- **Confirmation**: To confirm other indicators or signals
- **Multiple Timeframes**: For confluence analysis

### When to Avoid:
- **Strong Trends**: Can give false signals in trending markets
- **Low Volatility**: Less reliable in low volatility periods
- **As Standalone**: Should not be used as the only indicator
- **News Events**: Can be unreliable during high volatility

## 12. Advanced Stochastic Techniques

### 1. Stochastic with Multiple Timeframes
Use Stochastic on different timeframes to get a broader market perspective:
- **Long-term**: Stochastic(21, 5, 5) on daily charts for overall trend
- **Medium-term**: Stochastic(14, 3, 3) on 4-hour charts for intermediate signals
- **Short-term**: Stochastic(5, 3, 3) on 1-hour charts for entry timing

### 2. Stochastic and Moving Averages
Combine Stochastic with moving averages for better signals:
- **Trend Filter**: Only trade Stochastic signals in the direction of price MA
- **Stochastic MA Cross**: Signal when Stochastic crosses its own moving average

### 3. Stochastic Price Bands
Create dynamic overbought/oversold levels based on volatility:
- **Volatility Adjusted**: Adjust levels based on ATR
- **Regime Detection**: Different levels for trending vs. ranging markets

In [ ]:
# Advanced Stochastic Techniques
df_advanced = df_manual.copy()

# 1. Stochastic with Moving Averages
df_advanced['k_ma_9'] = df_advanced['k_14'].rolling(window=9).mean()
df_advanced['d_ma_9'] = df_advanced['d_14'].rolling(window=9).mean()

# 2. Stochastic Volatility Bands
df_advanced['k_std'] = df_advanced['k_14'].rolling(window=14).std()
df_advanced['k_upper_band'] = df_advanced['k_14'] + (df_advanced['k_std'] * 0.5)
df_advanced['k_lower_band'] = df_advanced['k_14'] - (df_advanced['k_std'] * 0.5)

# 3. Stochastic Rate of Change
df_advanced['k_roc'] = df_advanced['k_14'].pct_change() * 100
df_advanced['d_roc'] = df_advanced['d_14'].pct_change() * 100

# 4. Stochastic and Price Correlation
df_advanced['price_k_corr'] = df_advanced['close'].rolling(window=20).corr(df_advanced['k_14'])

# Plot advanced techniques
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Stochastic with Moving Averages
axes[0, 0].plot(df_advanced.index, df_advanced['k_14'], label='%K (14)', color='blue')
axes[0, 0].plot(df_advanced.index, df_advanced['d_14'], label='%D (3)', color='red')
axes[0, 0].plot(df_advanced.index, df_advanced['k_ma_9'], label='%K MA(9)', color='orange')
axes[0, 0].axhline(y=80, color='red', linestyle='--', alpha=0.5)
axes[0, 0].axhline(y=20, color='green', linestyle='--', alpha=0.5)
axes[0, 0].set_title('Stochastic with Moving Averages')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_ylim(0, 100)

# Stochastic Volatility Bands
axes[0, 1].plot(df_advanced.index, df_advanced['k_14'], label='%K (14)', color='blue')
axes[0, 1].plot(df_advanced.index, df_advanced['k_upper_band'], label='Upper Band', color='red', linestyle='--')
axes[0, 1].plot(df_advanced.index, df_advanced['k_lower_band'], label='Lower Band', color='green', linestyle='--')
axes[0, 1].fill_between(df_advanced.index, df_advanced['k_lower_band'], 
                        df_advanced['k_upper_band'], alpha=0.1, color='gray')
axes[0, 1].set_title('Stochastic Volatility Bands')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim(0, 100)

# Stochastic Rate of Change
axes[1, 0].plot(df_advanced.index, df_advanced['k_roc'], label='%K Rate of Change', color='blue')
axes[1, 0].plot(df_advanced.index, df_advanced['d_roc'], label='%D Rate of Change', color='red')
axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 0].fill_between(df_advanced.index, df_advanced['k_roc'], 0, alpha=0.3, color='blue')
axes[1, 0].set_title('Stochastic Rate of Change')
axes[1, 0].set_ylabel('Rate of Change (%)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Stochastic and Price Correlation
axes[1, 1].plot(df_advanced.index, df_advanced['price_k_corr'], label='Price-%K Correlation', color='blue')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Positive Correlation')
axes[1, 1].axhline(y=-0.5, color='red', linestyle='--', alpha=0.5, label='Negative Correlation')
axes[1, 1].set_title('Price-Stochastic Correlation')
axes[1, 1].set_ylabel('Correlation')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(-1, 1)

plt.tight_layout()
plt.show()

# Analysis of extreme Stochastic movements
print("Extreme Stochastic Movement Analysis:")
extreme_roc = df_advanced[abs(df_advanced['k_roc']) > 50][['close', 'k_14', 'k_roc']]

if len(extreme_roc) > 0:
    print(f"Found {len(extreme_roc)} instances of extreme Stochastic movements (>50% in one period)")
    display(extreme_roc.head())
else:
    print("No extreme Stochastic movements found in the dataset")

## 13. Risk Management with Stochastic

### 1. Position Sizing Based on Stochastic Levels
- **Extreme Levels**: Larger positions when Stochastic is at extremes
- **Moderate Levels**: Smaller positions when Stochastic is near 50
- **Divergence**: Increased position size on divergence signals

### 2. Stop Loss Placement
- **Oversold Buys**: Place stop loss below recent swing low
- **Overbought Sells**: Place stop loss above recent swing high
- **ATR-Based**: Use ATR to determine stop loss distance

### 3. Take Profit Levels
- **Stochastic Targets**: Take profit when Stochastic reaches opposite extreme
- **Scaling Out**: Partial profits at different Stochastic levels
- **Trailing Stops**: Use Stochastic levels for trailing stop adjustments

In [ ]:
# Risk Management with Stochastic
df_risk = df_manual.copy()

# Calculate ATR for volatility-based stops
df_risk['atr'] = IndicatorCalculator.atr(df_risk['high'], df_risk['low'], df_risk['close'], 14)

# Risk management techniques
df_risk['k_14'] = IndicatorCalculator.stochastic(
    df_risk['high'], df_risk['low'], df_risk['close'], 14, 3
)[0]

# 1. Position sizing based on Stochastic levels
df_risk['stoch_distance_from_center'] = abs(df_risk['k_14'] - 50)
df_risk['position_size'] = np.where(
    df_risk['stoch_distance_from_center'] > 30, 1.0,  # Large position at extremes
    np.where(
        df_risk['stoch_distance_from_center'] > 15, 0.5,  # Medium position
        0.25  # Small position near center
    )
)

# 2. Stop loss based on Stochastic levels and ATR
stop_loss_multiplier = 1.5
df_risk['stop_loss_long'] = df_risk['close'] - (stop_loss_multiplier * df_risk['atr'])
df_risk['stop_loss_short'] = df_risk['close'] + (stop_loss_multiplier * df_risk['atr'])

# 3. Risk-reward ratio based on Stochastic distance
df_risk['stoch_target_distance'] = abs(df_risk['k_14'] - 50)
df_risk['reward_per_unit'] = df_risk['stoch_target_distance'] * 0.3  # Scale factor
df_risk['risk_per_unit'] = df_risk['atr'] * stop_loss_multiplier
df_risk['risk_reward_ratio'] = df_risk['reward_per_unit'] / df_risk['risk_per_unit']

# Plot risk management
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Position sizing
axes[0, 0].plot(df_risk.index, df_risk['position_size'], label='Position Size', color='blue', linewidth=2)
axes[0, 0].fill_between(df_risk.index, df_risk['position_size'], alpha=0.3, color='blue')
axes[0, 0].set_title('Position Sizing Based on Stochastic Distance from Center')
axes[0, 0].set_ylabel('Position Size (Multiplier)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Stochastic levels and position size
axes[0, 1].scatter(df_risk['k_14'], df_risk['position_size'], alpha=0.6, c=df_risk['k_14'], cmap='RdYlGn')
axes[0, 1].axvline(x=20, color='green', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=80, color='red', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=50, color='gray', linestyle=':', alpha=0.5)
axes[0, 1].set_title('Position Size vs Stochastic Level')
axes[0, 1].set_xlabel('Stochastic')
axes[0, 1].set_ylabel('Position Size')
axes[0, 1].grid(True, alpha=0.3)

# Risk-reward ratio
axes[1, 0].plot(df_risk.index, df_risk['risk_reward_ratio'], label='Risk-Reward Ratio', color='orange')
axes[1, 0].axhline(y=2, color='red', linestyle='--', label='Minimum 2:1 Ratio')
axes[1, 0].fill_between(df_risk.index, df_risk['risk_reward_ratio'], 2, alpha=0.3, color='orange')
axes[1, 0].set_title('Risk-Reward Ratio Analysis')
axes[1, 0].set_ylabel('Ratio')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Stochastic distance distribution
axes[1, 1].hist(df_risk['stoch_distance_from_center'].dropna(), bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[1, 1].axvline(x=df_risk['stoch_distance_from_center'].quantile(0.7), color='red', linestyle='--', label='70th percentile')
axes[1, 1].axvline(x=df_risk['stoch_distance_from_center'].quantile(0.3), color='green', linestyle='--', label='30th percentile')
axes[1, 1].set_title('Stochastic Distance from Center Distribution')
axes[1, 1].set_xlabel('Distance from 50')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Risk management statistics
print("Risk Management Statistics:")
print(f"Average position size: {df_risk['position_size'].mean():.2f}")
print(f"Average risk-reward ratio: {df_risk['risk_reward_ratio'].mean():.2f}")
print(f"Trades with minimum 2:1 ratio: {(df_risk['risk_reward_ratio'] >= 2).sum()} ({(df_risk['risk_reward_ratio'] >= 2).mean()*100:.1f}%)")
print(f"Average Stochastic distance from center: {df_risk['stoch_distance_from_center'].mean():.1f}")

## 14. Conclusion

The Stochastic Oscillator is a powerful momentum indicator that provides valuable insights into market conditions. While it has limitations, particularly in trending markets, it remains one of the most reliable indicators for identifying overbought/oversold conditions and potential reversals.

### Key Takeaways:
1. **Momentum Measurement**: Stochastic excels at measuring the speed of price movements
2. **Extreme Conditions**: Excellent for identifying overbought/oversold levels
3. **Divergence Detection**: Early warning of potential trend reversals
4. **Crossover Signals**: Clear buy/sell signals from %K/%D crossovers
5. **Combination**: Works best when combined with trend indicators and volume analysis

### Best Practices:
- Use multiple Stochastic periods for confirmation
- Combine with trend indicators like moving averages
- Implement proper risk management techniques
- Consider market conditions when interpreting Stochastic signals
- Test different parameters to find what works best for your trading style

## 15. Exercises

### Exercise 1: Optimize Stochastic Parameters
Test different Stochastic parameters (K period, D period, slowing) to find the optimal combination for different market conditions. Create a function that tests multiple parameter combinations and returns the best performing one based on Sharpe ratio.

### Exercise 2: Create Stochastic Strategy with Volume Confirmation
Modify the Stochastic overbought/oversold strategy to include volume confirmation. Only enter trades when volume is above the 20-day average and Stochastic signals are confirmed.

### Exercise 3: Implement Stochastic with Multiple Timeframe Analysis
Create a strategy that uses Stochastic signals from multiple timeframes (e.g., daily, 4-hour, 1-hour) and only enters trades when signals align across timeframes.

### Solutions:
```python
# Exercise 1 Solution
def optimize_stochastic_parameters(data, k_range, d_range, slowing_range):
    best_sharpe = -float('inf')
    best_params = None
    
    for k_period in k_range:
        for d_period in d_range:
            for slowing in slowing_range:
                signals = stochastic_crossover_strategy(data, k_period, d_period, slowing)[['signal']]
                engine = BacktestEngine()
                result = engine.run_backtest(data, signals, f"Stoch_{k_period}_{d_period}_{slowing}")
                sharpe = result['metrics']['sharpe_ratio']
                
                if sharpe > best_sharpe:
                    best_sharpe = sharpe
                    best_params = (k_period, d_period, slowing)
    
    return best_params, best_sharpe

# Exercise 2 Solution
def stochastic_volume_strategy(data, k_period=14, d_period=3, slowing=3, volume_period=20):
    df = data.copy()
    
    # Calculate Stochastic and volume SMA
    df['k'], df['d'] = IndicatorCalculator.stochastic(
        df['high'], df['low'], df['close'], k_period, d_period
    )
    df['volume_sma'] = IndicatorCalculator.sma(df['volume'], volume_period)
    
    # Generate signals with volume confirmation
    df['signal'] = 0
    
    # Buy signal: Stochastic oversold AND high volume
    buy_condition = ((df['k'] <= 20) & 
                    (df['volume'] > df['volume_sma']))
    
    # Sell signal: Stochastic overbought AND high volume
    sell_condition = ((df['k'] >= 80) & 
                      (df['volume'] > df['volume_sma']))
    
    df.loc[buy_condition, 'signal'] = 1
    df.loc[sell_condition, 'signal'] = -1
    
    return df
```

## 16. Additional Resources

- **Books**:
  - "New Concepts in Technical Trading Systems" by J. Welles Wilder Jr.
  - "Technical Analysis of Financial Markets" by John J. Murphy
  - "Trading Systems and Methods" by Perry J. Kaufman

- **Online Resources**:
  - [TradingView Documentation](https://www.tradingview.com/pine-script-docs/)
  - [Investopedia - Stochastic Oscillator](https://www.investopedia.com/terms/s/stochasticoscillator.asp)
  - [Binance Academy - Technical Analysis](https://academy.binance.com/en/categories/technical-analysis)

- **Research Papers**:
  - "The Profitability of Technical Analysis in the Foreign Exchange Market" by Cheol-Ho Park and Scott H. Irwin
  - "Technical Analysis and Momentum Strategies in Commodity Futures" by Szakmary and Shen

- **Tools**:
  - [TradingView](https://www.tradingview.com/) - Charting and Pine Script
  - [Binance](https://www.binance.com/) - Trading and indicators
  - [Freqtrade](https://www.freqtrade.io/) - Algorithmic trading

---

**Disclaimer**: This tutorial is for educational purposes only. Trading cryptocurrencies involves significant risk. Always do your own research and consider consulting with a financial advisor before making investment decisions.